In [ ]:
!git clone https://github.com/tobecwb/stable-diffusion-regularization-images

Cloning into 'stable-diffusion-regularization-images'...
remote: Enumerating objects: 49166, done.
remote: Counting objects: 100% (10070/10070), done.
remote: Compressing objects: 100% (10041/10041), done.
remote: Total 49166 (delta 27), reused 10066 (delta 24), pack-reused 39096
Receiving objects: 100% (49166/49166), 10.02 GiB | 42.62 MiB/s, done.
Resolving deltas: 100% (50/50), done.
Updating files: 100% (30002/30002), done.


In [ ]:
!git clone https://github.com/bmaltais/kohya_ss.git
!wget https://raw.githubusercontent.com/yushan777/kohya_ss_vastai/main/!sdxl_kohya_vastai_no_config.ipynb
!mv '!sdxl_kohya_vastai_no_config.ipynb' 'kohya_ss'

Cloning into 'kohya_ss'...
remote: Enumerating objects: 10105, done.
remote: Counting objects: 100% (2378/2378), done.
remote: Compressing objects: 100% (253/253), done.
remote: Total 10105 (delta 2216), reused 2209 (delta 2122), pack-reused 7727
Receiving objects: 100% (10105/10105), 11.38 MiB | 15.29 MiB/s, done.
Resolving deltas: 100% (7006/7006), done.
--2023-12-25 15:19:23--  https://raw.githubusercontent.com/yushan777/kohya_ss_vastai/main/!sdxl_kohya_vastai_no_config.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16369 (16K) [text/plain]
Saving to: ‘!sdxl_kohya_vastai_no_config.ipynb’

!sdxl_kohya_vastai_ 100%[===================>]  15.99K  --.-KB/s    in 0.001s  

2023-12-25 15:19:24 (25.7 MB/s) - ‘!sdxl_kohya_vastai_no_config.ipynb’ saved [163

In [ ]:
!wget https://raw.githubusercontent.com/yushan777/kohya_ss_vastai/main/colors.py -q
from colors import bcolors
from IPython.display import clear_output

print(f"{bcolors.BOLD}{bcolors.GREEN}Installing prerequisites...{bcolors.ENDC}")
!apt-get update -y && apt-get install -y -qq libgl1
!sudo apt update -y && sudo apt install -y -qq python3-tk
clear_output(wait=True)

print(f"{bcolors.BOLD}{bcolors.GREEN}Installing main dependencies - this will take a few minutes...{bcolors.ENDC}")

%pip install gdown
!chmod +x setup.sh
!./setup.sh

print(f"{bcolors.BOLD}{bcolors.GREEN}Finished installing dependencies.{bcolors.ENDC}")

Installing main dependencies - this will take a few minutes...
chmod: cannot access 'setup.sh': No such file or directory
/bin/bash: line 1: ./setup.sh: No such file or directory
Finished installing dependencies.


In [ ]:
# 2. Configure Accelerate With defaults and bf16 precision
!accelerate config default --mixed_precision "fp16"

Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


In [ ]:
# 3. VARIABLES
# The following variables can be changed according to your project preferences
token_word = "ohwx"
class_word = "woman"
training_repeats = 40
training_root_dir = "training_images"
regularization_root_dir = "reg_images"
models_dir = "training_models"
project_name = "myProject"
output_dir = "trained_models"

In [ ]:
# 4. CREATE DATASET FOLDERS & MODEL FOLDER

# ================================================================
# your training and reg image subfolders will be named according to what is set in previous cell:
# so if you stuck with the defaults, they would be for example :
# "training_images/40_ohwx person"
# "reg_images/1_person"
training_dir = f'{training_root_dir}/{training_repeats}_{token_word} {class_word}'
reg_dir = f'{regularization_root_dir}/1_{class_word}'

# back to parent folder
%cd /workspace/kohya_ss

import os
if os.path.exists(training_dir) == False:
  os.makedirs(training_dir)
  print(f'{training_dir} Created.')
else:
  print(f'{training_dir} already exists.')

if os.path.exists(reg_dir) == False:
  os.makedirs(reg_dir)
  print(f'{reg_dir} Created.')
else:
  print(f'{reg_dir} already exists.')

if os.path.exists(models_dir) == False:
  os.makedirs(models_dir)
  print(f'{models_dir} Created.')
else:
  print(f'{models_dir} already exists.')

# Create prompt file use for samples during training.
# first 2 prompts are the same but one has CFG=1 and the other CFG=7
# if subject likeness is strong and regular even at CFG=1 then it is an good indicator that it is overfitted
# third prompt is to check how well it responds to styling.
lines = [
    f"a photo of {token_word} {class_word} --w 512 --h 512 --l 7, --s 20 --d 1234567890\n",
    f"a photo of {token_word} {class_word} --w 512 --h 512 --l 1, --s 20 --d 1234567890\n",
    f"a portrait of {token_word} {class_word} in the style of Rembrandt --w 512 --h 512 --l 6, --s 20 --d 1234567890\n"
]

# Create and write to the text file
with open("prompt.txt", "w") as file:
    file.writelines(lines)

print("File 'prompt.txt' has been created.")

[Errno 2] No such file or directory: '/workspace/kohya_ss'
/content/kohya_ss
training_images/40_ohwx woman Created.
reg_images/1_woman Created.
training_models already exists.
File 'prompt.txt' has been created.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
training_dir

'training_images/40_ohwx woman'

In [ ]:
import shutil
import os

# Define the source directory (Google Drive)
source_dir = '/content/drive/MyDrive/DL_Project/ND_squared'

# Define the destination directory (local directory in Colab)
destination_dir = '/content/training_images/40_ohwx woman'  # Replace with your desired local path

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Copy each file from the source directory to the destination directory
for filename in os.listdir(source_dir):
    file_path = os.path.join(source_dir, filename)
    if os.path.isfile(file_path):
        shutil.copy(file_path, destination_dir)

print("Files copied successfully.")


Files copied successfully.


In [ ]:
reg_dir

'reg_images/1_woman'

In [ ]:
import os

# Define the directory path
directory_path = '/content/stable-diffusion-regularization-images/sdxl/woman'

# Check if the directory exists
if os.path.exists(directory_path):
    # List all files in the directory
    files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

    # Count the number of files
    file_count = len(files)

    print(f"Number of files in '{directory_path}': {file_count}")
else:
    print(f"The directory '{directory_path}' does not exist.")


Number of files in '/content/stable-diffusion-regularization-images/sdxl/woman': 5000


In [ ]:
import os

# Define the directory path
directory_path = '/content/drive/MyDrive/DL_Project/ND_squared'

# Check if the directory exists
if os.path.exists(directory_path):
    # List all files in the directory
    files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

    # Count the number of files
    file_count = len(files)

    print(f"Number of files in '{directory_path}': {file_count}")
else:
    print(f"The directory '{directory_path}' does not exist.")


Number of files in '/content/drive/MyDrive/DL_Project/ND_squared': 96


In [ ]:
import shutil
import os

# Define the source directory (Google Drive)
source_dir = '/content/stable-diffusion-regularization-images/sdxl/woman'

# Define the destination directory (local directory in Colab)
destination_dir = '/content/reg_images/1_woman'  # Replace with your desired local path

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Copy each file from the source directory to the destination directory
for filename in os.listdir(source_dir):
    file_path = os.path.join(source_dir, filename)
    if os.path.isfile(file_path):
        shutil.copy(file_path, destination_dir)

print("Files copied successfully.")


Files copied successfully.


In [ ]:
import shutil
import os

# Define the source directory (Google Drive)
source_dir = '/content/stable-diffusion-regularization-images/sdxl/woman'

# Define the destination directory (local directory in Colab)
destination_dir = '/content/drive/MyDrive/DL_Project/SDXL_5k_wmn'  # Replace with your desired local path

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Copy each file from the source directory to the destination directory
for filename in os.listdir(source_dir):
    file_path = os.path.join(source_dir, filename)
    if os.path.isfile(file_path):
        shutil.copy(file_path, destination_dir)

print("Files copied successfully.")


Files copied successfully.


In [ ]:
# 7. Check Dataset Folders
# ====================================================
# remove any non-image files & warn if any additional folders exist
import os
import shutil
from glob import glob
folder_path = f'{training_dir}'

# Get a list of all files in the folder
files = glob(folder_path + '/*', recursive=False)

# Iterate over the files and delete the ones that are not JPG or PNG
for file_path in files:
    if not (file_path.endswith('.jpg') or file_path.endswith('.png')):
        if os.path.isfile(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            print(f'{bcolors.BOLD}{bcolors.RED} Unexpected folder: \'{file_path}\' was found in training images folder path.  Check and remove it.{bcolors.ENDC}')

# force remove hidden .ipynb_checkpoints folder in images folder.
if os.path.exists(f'{folder_path}/.ipynb_checkpoints'):
    shutil.rmtree(f'{folder_path}/.ipynb_checkpoints')

# ====================================================
# delete any non-image files & warn if any additional folders

folder_path = f'{reg_dir}'

# Get a list of all files in the folder
files = glob(folder_path + '/*', recursive=False)

# Iterate over the files and delete the ones that are not JPG or PNG
for file_path in files:
    if not (file_path.endswith('.jpg') or file_path.endswith('.png')):
        if os.path.isfile(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            print(f'{bcolors.BOLD}{bcolors.RED} Unexpected folder: \'{file_path}\' was found in training images folder path.  Check and remove it.{bcolors.ENDC}')

# force remove hidden .ipynb_checkpoints folder in images folder.
if os.path.exists(f'{folder_path}/.ipynb_checkpoints'):
    shutil.rmtree(f'{folder_path}/.ipynb_checkpoints')

In [ ]:
# 8. Download SDXL 1.0 (0.9VAE) base model (the one with 0.9vae baked in instead of 1.0vae
from huggingface_hub import hf_hub_download

# Specify the repository ID, the filename, and the desired download directory
repo_id = 'stabilityai/stable-diffusion-xl-base-1.0'
filename = 'sd_xl_base_1.0_0.9vae.safetensors'
download_directory = '/content/training_models'

# Download the file to the training_models dir, as a regular file (not symlink)
base_model = hf_hub_download(repo_id=repo_id, filename=filename, local_dir=download_directory, local_dir_use_symlinks=False)
print(base_model)


sd_xl_base_1.0_0.9vae.safetensors:   0%|          | 0.00/6.94G [00:00<?, ?B/s]

/content/training_models/sd_xl_base_1.0_0.9vae.safetensors


In [ ]:
base_model, training_root_dir, regularization_root_dir, output_dir

('/content/training_models/sd_xl_base_1.0_0.9vae.safetensors',
 'training_images',
 'reg_images',
 'trained_models')

In [ ]:
%pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 9.0

In [ ]:
%cd /content/kohya_ss

/content/kohya_ss


In [ ]:
!pip install -qq diffusers==0.16.1 transformers ftfy accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.9/934.9 kB 7.4 MB/s eta 0:00:00


In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.6 MB/s eta 0:00:00


In [ ]:
!pip install voluptuous

In [ ]:
import os
import shutil

# Define the source and destination directories
source_dir = '/content/training_images/40_ohwx woman'
destination_dir = '/content/training_images'

# Check if both the source and destination directories exist
if os.path.exists(source_dir) and os.path.exists(destination_dir):
    # Iterate over all files in the source directory
    for filename in os.listdir(source_dir):
        file_path = os.path.join(source_dir, filename)

        # Check if it's a file and not a directory
        if os.path.isfile(file_path):
            # Move each file to the destination directory
            shutil.move(file_path, destination_dir)

    print("All files moved successfully.")
else:
    print("Check if both source and destination directories exist.")


All files moved successfully.


In [ ]:
!pwd

/content/kohya_ss


In [ ]:
import os
import glob

# Define the root directory to start the search
root_dir = './'

# List to store the paths of found .toml files
toml_files = []

# Walk through the directory and its subdirectories
for dirpath, dirnames, filenames in os.walk(root_dir):
    # Use glob to find .toml files in the current directory
    for filename in glob.glob(os.path.join(dirpath, '*.toml')):
        toml_files.append(filename)

# Print the list of found .toml files
for file in toml_files:
    print(file)


./pyproject.toml
./_typos.toml


In [ ]:
import os
import shutil

# Define the source and destination directories
source_dir = '/content/kohya_ss/training/reg_images'
destination_dir = '/content/kohya_ss/training/reg_images/1_woman'

# Check if both the source and destination directories exist
if not os.path.exists(destination_dir):
    print(f"The destination directory {destination_dir} does not exist.")
elif not os.path.exists(source_dir):
    print(f"The source directory {source_dir} does not exist.")
else:
    # Iterate over all files in the source directory
    for filename in os.listdir(source_dir):
        # Construct the full file path
        file_path = os.path.join(source_dir, filename)
        # Check if it's a file and not a directory and if it's an image
        if os.path.isfile(file_path) and file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
            # Move each image to the destination directory
            shutil.move(file_path, destination_dir)

    print("All images have been moved to the 40_ohwx_woman folder.")


All images have been moved to the 40_ohwx_woman folder.


In [ ]:
!accelerate config default

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
max_steps = 7500
learning_rate = 1e-5
learning_rate_te1 = 1e-7
learning_rate_te2 = 1e-7
save_every_n_steps = 1000

print(base_model)
!accelerate launch --num_cpu_threads_per_process=2 "/content/kohya_ss/sdxl_train.py" \
  --pretrained_model_name_or_path="/content/training_models/sd_xl_base_1.0_0.9vae.safetensors" \
  --train_data_dir="/content/kohya_ss/training/training_images" \
  --reg_data_dir="/content/kohya_ss/training/reg_images" \
  --output_dir="/content/kohya_ss/trained_models" \
  --output_name="{project_name}" \
  --save_model_as="safetensors" \
  --train_batch_size=2 \
  --max_train_steps={max_steps} \
  --save_every_n_steps={save_every_n_steps} \
  --optimizer_type="adafactor" \
  --optimizer_args scale_parameter=False relative_step=False warmup_init=False \
  --xformers \
  --cache_latents \
  --lr_scheduler="constant_with_warmup" \
  --lr_warmup_steps=100 \
  --learning_rate="{learning_rate}" \
  --learning_rate_te1="{learning_rate_te1}" \
  --learning_rate_te2="{learning_rate_te2}" \
  --max_grad_norm=0.0 \
  --train_text_encoder \
  --resolution="512,512" \
  --save_precision="fp16" \
  --save_n_epoch_ratio=1 \
  --max_data_loader_n_workers=1 \
  --persistent_data_loader_workers \
  --mixed_precision="fp16" \
  --full_bf16 \
  --logging_dir="logs" \
  --log_prefix="last" \
  --gradient_checkpointing \
  --min_snr_gamma=0 \
  --noise_offset=0.0357 \
  --sample_sampler="euler_a" \
  --sample_prompts="prompt.txt" \
  --sample_every_n_steps=100

# Training finished...
# List all checkpoints
print("Trained models:\n")
!ls trained_models -1

/content/training_models/sd_xl_base_1.0_0.9vae.safetensors
2023-12-25 16:58:04.590835: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-25 16:58:04.590892: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-25 16:58:04.592252: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-25 16:58:06.028239: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
prepare tokenizers
Using DreamBooth method.
ignore directory without repeats / 繰り返し回数のないディレクトリを無視します: .ipynb_checkpoints
ignore directory without repeats / 繰り返し回数のないディレクトリを無視します: .ip